In [1]:
import sys
from transformers import ( 
        Trainer,
        AutoModelForSequenceClassification,
        AutoConfig)
import objsize


/Users/DP15886/Documents/cds/introml/ml-project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from utils.model import load_models
from utils.quantizer import quantize_model, norm_quantize

In [12]:
def count_memory(model_loaded, n_bits_base):
    total_non_zero_params = sum(p.count_nonzero() for p in model_loaded.parameters())
    total_params = sum(p.numel() for p in model_loaded.parameters())
    total_zeros = total_params-total_non_zero_params
    # print(total_non_zero_params)
    # print(total_zeros)
    n_bits = 32
    return total_non_zero_params* (n_bits/8)*1e-6+ (1e-6*total_zeros/8)

In [13]:
task_to_keys = {
    'cola': ('sentence', None),
    'mnli': ('premise', 'hypothesis'),
    'mrpc': ('sentence1', 'sentence2'),
    'qnli': ('question', 'sentence'),
    'qqp': ('question1', 'question2'),
    'rte': ('sentence1', 'sentence2'),
    'sst2': ('sentence', None),
    'stsb': ('sentence1', 'sentence2'),
    'wnli': ('sentence1', 'sentence2')}


ms = [
'JeremiahZ/bert-base-uncased-cola',
'doyoungkim/bert-base-uncased-finetuned-sst2',
'Intel/bert-base-uncased-mrpc',
'JeremiahZ/bert-base-uncased-stsb',
'JeremiahZ/bert-base-uncased-qqp',
'JeremiahZ/bert-base-uncased-mnli',
'JeremiahZ/bert-base-uncased-qnli',
'JeremiahZ/bert-base-uncased-rte',
'JeremiahZ/bert-base-uncased-wnli']

n_bits_base = 32

for m in ms:
    model_loaded = AutoModelForSequenceClassification.from_pretrained(m)
    print("Non-quantized memory: {:.2f} MB".format(count_memory(model_loaded, n_bits_base)))
    model_quantized = quantize_model(model_loaded, lambda X: norm_quantize(X, 0.01))
    
    print("Quantized memory: {:.2f} MB".format(count_memory(model_quantized, n_bits_base)))

Non-quantized memory: 437.94
Quantized memory: 351.62 MB
Non-quantized memory: 437.94
Quantized memory: 351.58 MB
Non-quantized memory: 437.94
Quantized memory: 351.62 MB
Non-quantized memory: 437.93
Quantized memory: 351.61 MB
Non-quantized memory: 437.94
Quantized memory: 351.74 MB
Non-quantized memory: 437.94
Quantized memory: 351.73 MB
Non-quantized memory: 437.94
Quantized memory: 351.64 MB
Non-quantized memory: 437.94
Quantized memory: 351.61 MB
Non-quantized memory: 437.94
Quantized memory: 351.61 MB
